In [1]:
import os
import glob

import cv2
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

/opt/homebrew/Caskroom/miniconda/base/envs/synthesis-car-od/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def transforms():
    return A.Compose([
        A.Normalize(mean=(0, 0, 0), std =(1, 1, 1)),
        ToTensorV2(),
    ])

def grayscale_transforms():
    return A.Compose([
        A.ToGray(p=1),
        A.Normalize(mean=(0, 0, 0), std =(1, 1, 1)),
        ToTensorV2(),
    ])

In [3]:
class CustomDataset(Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = sorted(glob.glob(root+'/*.png'))
        
    def __getitem__(self, idx):
        img = cv2.imread(self.imgs[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.transforms is not None:
            transformed = self.transforms(image=img)
            img = transformed["image"]

        return img

    def __len__(self):
        return len(self.imgs)

In [4]:
working_dir = 'synthesis-car-od'
dir_len = len(working_dir)

path = os.getcwd().replace('\\', '/')
index = path.find(working_dir)
working_dir = path[:index + dir_len + 1]

In [5]:
DATA_PATH = os.path.join(working_dir, 'data/')
DATA_PATH

'/Users/a16/Desktop/JJ/self_study/projects/synthesis-car-od/data/'

In [6]:
rgb_dataset = CustomDataset(os.path.join(DATA_PATH, 'images/'), transforms=transforms())
rgb_loader = DataLoader(rgb_dataset, batch_size=128, shuffle=False)

gray_dataset = CustomDataset(os.path.join(DATA_PATH, 'images/'), transforms=grayscale_transforms())
gray_loader = DataLoader(gray_dataset, batch_size=128, shuffle=False)

### RGB Mean & STD

In [7]:
psum = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs in tqdm(rgb_loader):
    psum += inputs.sum(axis=[0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis=[0, 2, 3])

100%|██████████| 51/51 [08:02<00:00,  9.47s/it]


In [8]:
# pixel count
count = len(rgb_dataset) * 1920 * 1040

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: ' + str(total_mean))
print('std:  ' + str(total_std))

mean: tensor([0.4370, 0.4336, 0.4148])
std:  tensor([0.2531, 0.2283, 0.1971])


### Grayscale Mean & Std

In [9]:
psum = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs in tqdm(gray_loader):
    psum += inputs.sum(axis=[0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis=[0, 2, 3])

100%|██████████| 51/51 [08:01<00:00,  9.45s/it]


In [11]:
# pixel count
count = len(gray_dataset) * 1920 * 1040

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: ' + str(total_mean))
print('std:  ' + str(total_std))

mean: tensor([0.4325, 0.4325, 0.4325])
std:  tensor([0.2300, 0.2300, 0.2300])
